# Cleaning Data

In [104]:
import time
#from selenium import webdriver
from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import re
import os
from lxml import etree
import lxml.html, lxml.html.clean
from urllib.parse import urlparse
from urllib.parse import parse_qs
import pandas as pd
from itertools import product
import time
import regex
import numpy as np

#driver = webdriver.Chrome(ChromeDriverManager().install())
verbose_flag = True
# htmlSource = driver.page_source

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

### Load data


In [91]:
path_data_scientist = r"C:\Users\RSYeo\Projects\data\data_scientist2_2022_03_02.ftr"
path_data_engineer =r"C:\Users\RSYeo\Projects\data\data_engineer_2022_03_02.ftr"
path_data_analyst = r"C:\Users\RSYeo\Projects\data\data_analyst_2022_03_02.ftr"

In [92]:
df_data_scientist = pd.read_feather(path_data_scientist)
df_data_scientist.head()

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state
0,[jobstreet-my-job-4844260],"Policy, Strategy & Credit Planning (Data Scien...",Posted on 28-Feb-22,Job DescriptionJob Summary:Lead and manage cre...,Manager,5 years,Full-Time,"Diploma, Advanced/Higher/Graduate Diploma, Bac...","Accounting/Finance, Banking/Financial",Hong Leong Bank Berhad,97141-X,More than 5000 Employees,Banking / Financial Services,/en/job/policy-strategy-credit-planning-data-s...,data scientist,selangor
1,[jobstreet-my-job-4831428],Data Scientist,Posted on 24-Feb-22,Job DescriptionJob Responsibilities:Responsibl...,Manager,4 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",Credit Bureau Malaysia Sdn Bhd,820050-T,1 - 50 Employees,Banking / Financial Services,/en/job/data-scientist-4831428?jobId=jobstreet...,data scientist,selangor
2,[jobstreet-my-job-4848448],Data Scientist,Posted on 25-Feb-22,Job HighlightsNiche Market Player in Financial...,Junior Executive,2 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",TESS International Group,508613-A,51 - 200 Employees,Computer / Information Technology (Software),/en/job/data-scientist-4848448?jobId=jobstreet...,data scientist,selangor
3,[jobstreet-my-job-4854027],Data Science for Region Overseas Experience Ce...,Posted 7 hours ago,Job HighlightsA competitive package within a l...,Manager,4 years,Full-Time,Not Specified,"Computer/Information Technology, IT-Network/Sy...",Mercedes-Benz Malaysia Sdn Bhd,596096-H,201 - 500 Employees,Automobile / Automotive Ancillary / Vehicle,/en/job/data-science-for-region-overseas-exper...,data scientist,selangor
4,[jobstreet-my-job-4848435],Data Scientist Manager,Posted on 25-Feb-22,Job DescriptionRESPONSIBILITIESResponsible for...,Manager,3 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Sales/Marketing, Digital Marketing",Hong Leong Assurance Berhad,94613-X,501 - 1000 Employees,Insurance,/en/job/data-scientist-manager-4848435?jobId=j...,data scientist,selangor


### Start cleaning data

#### Format ID

In [93]:
# Format job id to be numerical
df_data_scientist['job_id'] = df_data_scientist.job_id.astype('str').str.extract("(\d+)")

In [94]:
df_data_scientist.job_id.value_counts()

4844260       1
1030693812    1
1030761270    1
1030425803    1
1030550709    1
             ..
4844693       1
4840131       1
4842253       1
4818476       1
1030436103    1
Name: job_id, Length: 745, dtype: int64

In [95]:
df_data_scientist[df_data_scientist['job_id'] == '9316926'].iloc[1:5]

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state


In [96]:
df_data_scientist[df_data_scientist.state == 'selangor'].reset_index()

,index,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state
0,0,4844260,"Policy, Strategy & Credit Planning (Data Scien...",Posted on 28-Feb-22,Job DescriptionJob Summary:Lead and manage cre...,Manager,5 years,Full-Time,"Diploma, Advanced/Higher/Graduate Diploma, Bac...","Accounting/Finance, Banking/Financial",Hong Leong Bank Berhad,97141-X,More than 5000 Employees,Banking / Financial Services,/en/job/policy-strategy-credit-planning-data-s...,data scientist,selangor
1,1,4831428,Data Scientist,Posted on 24-Feb-22,Job DescriptionJob Responsibilities:Responsibl...,Manager,4 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",Credit Bureau Malaysia Sdn Bhd,820050-T,1 - 50 Employees,Banking / Financial Services,/en/job/data-scientist-4831428?jobId=jobstreet...,data scientist,selangor
2,2,4848448,Data Scientist,Posted on 25-Feb-22,Job HighlightsNiche Market Player in Financial...,Junior Executive,2 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",TESS International Group,508613-A,51 - 200 Employees,Computer / Information Technology (Software),/en/job/data-scientist-4848448?jobId=jobstreet...,data scientist,selangor
3,3,4854027,Data Science for Region Overseas Experience Ce...,Posted 7 hours ago,Job HighlightsA competitive package within a l...,Manager,4 years,Full-Time,Not Specified,"Computer/Information Technology, IT-Network/Sy...",Mercedes-Benz Malaysia Sdn Bhd,596096-H,201 - 500 Employees,Automobile / Automotive Ancillary / Vehicle,/en/job/data-science-for-region-overseas-exper...,data scientist,selangor
4,4,4848435,Data Scientist Manager,Posted on 25-Feb-22,Job DescriptionRESPONSIBILITIESResponsible for...,Manager,3 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Sales/Marketing, Digital Marketing",Hong Leong Assurance Berhad,94613-X,501 - 1000 Employees,Insurance,/en/job/data-scientist-manager-4848435?jobId=j...,data scientist,selangor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,740,1030448849,Laboratory Technician,Posted on 4-Feb-22,Job DescriptionJob ResponsibilityJob descripti...,Not Specified,None,Full-Time,Not Specified,"Manufacturing, Manufacturing",MIE AGRO FARM,433722-P,501 - 1000 Employees,Construction / Building / Engineering,/en/job/laboratory-technician-1030448849?jobId...,data scientist,selangor
741,741,1030436271,Technical Specialist,Posted on 3-Feb-22,Job DescriptionJob: TechnologyPrimary Location...,Not Specified,None,Full-Time,Not Specified,"Computer/Information Technology, IT-Network/Sy...",Standard Chartered,115793-P,More than 5000 Employees,Banking / Financial Services,/en/job/technical-specialist-1030436271?jobId=...,data scientist,selangor
742,742,1030436231,Senior System Specialist (DHL Express),Posted on 3-Feb-22,Job DescriptionWouldyou like to be part of the...,Not Specified,None,None,Not Specified,"Computer/Information Technology, IT-Software",DHL Supply Chain (Malaysia) Sdn Bhd,7908-W,501 - 1000 Employees,Transportation / Logistics,/en/job/senior-system-specialist-dhl-express-1...,data scientist,selangor
743,743,1030436028,Senior Software Engineer (Angular),Posted on 3-Feb-22,Job DescriptionSUMMARYBegin your career with D...,Not Specified,None,None,Not Specified,"Computer/Information Technology, IT-Software",DHL Supply Chain (Malaysia) Sdn Bhd,7908-W,501 - 1000 Employees,Transportation / Logistics,/en/job/senior-software-engineer-angular-10304...,data scientist,selangor


In [72]:
df_data_scientist[df_data_scientist['job_id'] == 'jobstreet-my-job-4839088']

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state,job_id2


In [73]:
df_data_scientist.head()

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state,job_id2
0,NaN,Big Data Lead (Work in Singapore) - 44565,"SGD 7,000 - SGD 10,500",Job HighlightsExciting future with breakthroug...,Manager,7 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",Keysight Technologies Malaysia Sdn. Bhd.,463532-M,2001 - 5000 Employees,Electrical & Electronics,/en/job/big-data-lead-work-in-singapore-44565-...,data scientist,negeri-sembilan,['jobstreet-my-job-4839088']
1,NaN,SQL Database Administrator (数据库管理员),"MYR 15,000 - MYR 25,000","Job DescriptionJob Requirements: Must speak, r...",Senior Executive,3 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Network/Sy...",Morivy Data and Technology Inc.,None,1 - 50 Employees,Computer / Information Technology (Software),/en/job/sql-database-administrator-%E6%95%B0%E...,data scientist,negeri-sembilan,['jobstreet-my-job-4839088']
2,['jobstreet-my-job-4839088'],Korean Content Specialist (Pharmaceutical) Bas...,Posted on 24-Feb-22,Job HighlightsBest Workplace EverCompetitive B...,Senior Executive,1 year,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Others, Publishing",Clarivate (Malaysia) Sdn Bhd,1360469-D,2001 - 5000 Employees,"Consulting (IT, Science, Engineering & Technical)",/en/job/korean-content-specialist-pharmaceutic...,data scientist,negeri-sembilan,['jobstreet-my-job-4839088']
3,['jobstreet-my-job-4839088'],Manufacturing Engineer- Fresh Graduates (1 yea...,"MYR 3,000 - MYR 3,500",Job HighlightsCareer AdvancementDaily Meal All...,Entry Level,None,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Engineering, Electrical, Mechanical",Plexus Manufacturing Sdn Bhd,399136-M,More than 5000 Employees,Electrical & Electronics,/en/job/manufacturing-engineer-fresh-graduates...,data scientist,negeri-sembilan,['jobstreet-my-job-4839088']
4,['jobstreet-my-job-4839088'],Production Executive,Posted 12 hours ago,Job DescriptionJob Summary Responsible for the...,Senior Executive,3 years,Full-Time,"Diploma, Advanced/Higher/Graduate Diploma, Bac...","Manufacturing, Manufacturing",SUNSHINE BREAD SDN BHD (Formerly known as Auri...,1219557K,51 - 200 Employees,Manufacturing / Production,/en/job/production-executive-4852655?jobId=job...,data scientist,negeri-sembilan,['jobstreet-my-job-4839088']


#### Format salary range

In [102]:
#replace wrong values with nan
df_data_scientist.loc[df_data_scientist['salary_range'].str.contains('Posted'), 'salary_range'] = np.nan

In [119]:
df_malayysian_salary = df_data_scientist.loc[~df_data_scientist['salary_range'].isnull() & df_data_scientist['salary_range'].str.contains('MYR'), :]
df_malayysian_salary.

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state
10,4842705,"Assistant Manager, Data Analytics & Management","MYR 5,400 - MYR 7,000",Job HighlightsCareer development and opportuni...,Manager,5 years,Full-Time,Not Specified,"Computer/Information Technology, IT-Network/Sy...",South East Asia Iron And Steel Institute,None,1 - 50 Employees,Non-Profit Organisation / Social Services / NGO,/en/job/assistant-manager-data-analytics-manag...,data scientist,selangor
13,4809824,Data Engineer,"MYR 5,000 - MYR 9,000","Job DescriptionAs a Data Engineer, you will bu...",Senior Executive,5 years,Full-Time,"Diploma, Advanced/Higher/Graduate Diploma, Bac...","Computer/Information Technology, IT-Network/Sy...",PPG Coatings (Malaysia) Sdn Bhd,None,51 - 200 Employees,Manufacturing / Production,/en/job/data-engineer-4809824?jobId=jobstreet-...,data scientist,selangor
14,4853752,Senior Software Developer (Python Preferable),"MYR 7,000 - MYR 10,000",Job HighlightsGood Career OpportunitiesGood Id...,Senior Executive,5 years,Full-Time,"Diploma, Advanced/Higher/Graduate Diploma, Bac...","Computer/Information Technology, IT-Software",HaloCheck Sdn Bhd,1392729-M,1 - 50 Employees,Human Resources Management / Consulting,/en/job/senior-software-developer-python-prefe...,data scientist,selangor
17,4852082,Corporate Insights Analyst,"MYR 3,300 - MYR 5,000",Job DescriptionWHAT ARE THE JOB FUNCTIONS?Meet...,Senior Executive,3 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Sciences, Actuarial/Statistics",iWisers,201701016706,51 - 200 Employees,"Consulting (IT, Science, Engineering & Technical)",/en/job/corporate-insights-analyst-4852082?job...,data scientist,selangor
18,4852405,Big Data Platform Administrator,"MYR 5,000 - MYR 7,000",Job HighlightsOutstanding career development o...,Senior Executive,2 years,Full-Time,"Diploma, Advanced/Higher/Graduate Diploma, Bac...","Computer/Information Technology, IT-Network/Sy...",Merchantrade Asia Sdn Bhd,410591-T,501 - 1000 Employees,Banking / Financial Services,/en/job/big-data-platform-administrator-485240...,data scientist,selangor
